<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW2/blob/main/BigData_HW2_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [2]:
import pandas as pd
import numpy as np

# Dataset

In [3]:
!gdown 1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
!unzip /content/Bigdata_hw2_datasets.zip

Downloading...
From: https://drive.google.com/uc?id=1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
To: /content/Bigdata_hw2_datasets.zip
100% 6.28M/6.28M [00:00<00:00, 27.1MB/s]
Archive:  /content/Bigdata_hw2_datasets.zip
   creating: Bigdata_hw2_datasets/
   creating: Bigdata_hw2_datasets/q1/
  inflating: Bigdata_hw2_datasets/q1/stream_data_dgim.txt  
   creating: Bigdata_hw2_datasets/q2/
  inflating: Bigdata_hw2_datasets/q2/games.csv  
  inflating: Bigdata_hw2_datasets/q2/ratings.csv  
   creating: Bigdata_hw2_datasets/q3/
  inflating: Bigdata_hw2_datasets/q3/c1.txt  
  inflating: Bigdata_hw2_datasets/q3/c2.txt  
  inflating: Bigdata_hw2_datasets/q3/data.txt  


In [4]:
games_dataset = pd.read_csv("/content/Bigdata_hw2_datasets/q2/games.csv")

In [5]:
games_dataset = games_dataset[games_dataset["game_id"].notna()]

In [6]:
ratings_dataset = pd.read_csv("/content/Bigdata_hw2_datasets/q2/ratings.csv")

# Recommendation Functions

In [9]:
def rated_item_list(user_id):
  return ratings_dataset[ratings_dataset["user_id"]==user_id]["game_id"].values

In [10]:
def rater_user_list(item_id):
  return ratings_dataset[ratings_dataset["game_id"]==item_id]["user_id"].values

In [11]:
def mean_rate_user(user_id):
  return ratings_dataset[ratings_dataset["user_id"]==user_id]["rating"].mean()

In [12]:
def mean_rate_item(item_id):
  return ratings_dataset[ratings_dataset["game_id"]==item_id]["rating"].mean()

In [13]:
def rate(user_id, item_id):
  return ratings_dataset[(ratings_dataset["user_id"]==user_id) & (ratings_dataset["game_id"]==item_id)]["rating"].values[0]

In [20]:
def similarity_users(user_id_1, user_id_2):
  common_item_list = np.intersect1d(rated_item_list(user_id_1), rated_item_list(user_id_2))

  numerator = 0
  denominator_1 = 0
  denominator_2 = 0
  
  mean_rate_user_1 = mean_rate_user(user_id_1)
  mean_rate_user_2 = mean_rate_user(user_id_2)

  if len(common_item_list) == 0:
    return None

  for item_id in common_item_list:
    numerator += (rate(user_id_1, item_id)-mean_rate_user_1) * (rate(user_id_2, item_id)-mean_rate_user_2)    

    denominator_1 += (rate(user_id_1, item_id)-mean_rate_user_1) ** 2
    denominator_2 += (rate(user_id_2, item_id)-mean_rate_user_2) ** 2
    
  return numerator / ((denominator_1 ** 0.5) * (denominator_2 ** 0.5))

In [23]:
def similarity_items(item_id_1, item_id_2):
  common_user_list = np.intersect1d(rater_user_list(item_id_1), rater_user_list(item_id_2))

  numerator = 0
  denominator_1 = 0
  denominator_2 = 0
  
  mean_rate_item_1 = mean_rate_item(item_id_1)
  mean_rate_item_2 = mean_rate_item(item_id_2)

  if len(common_user_list) == 0:
    return None

  for user_id in common_user_list:
    numerator += (rate(user_id, item_id_1)-mean_rate_item_1) * (rate(user_id, item_id_2)-mean_rate_item_2)    

    denominator_1 += (rate(user_id, item_id_1)-mean_rate_item_1) ** 2
    denominator_2 += (rate(user_id, item_id_2)-mean_rate_item_2) ** 2
    
  return numerator / ((denominator_1 ** 0.5) * (denominator_2 ** 0.5))

# Results

In [27]:
similarity_items(1, 5)

0.09819797153379713